In [1]:
from tensorflow import keras

import tensorflow as tf
import matplotlib.pyplot as plt 
import xarray as xr
import glob
import numpy as np
#import 
%load_ext tensorboard

In [2]:
# Hyperparameters useful stuff - 
num_epochs = 4
batch_size = 32
#learning_rate = 0.001

DATA_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'
MODEL_STORE_PATH = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/storedModel/'

In [3]:
def load_data(start = "2012-01-01", stop = "2019-01-01", season = "SON", 
              era_path = '/uio/lagringshotell/geofag/students/metos/hannasv/era_interim_data/'):
    
    era_path = '/home/hanna/lagrings/era_interim_data/'
    """ Want all year put season to. """
    assert len(glob.glob(era_path + "*_q_*" + season +".nc")), "no data run cnct_lh in bash"
    #print(
    q   = xr.open_dataset(glob.glob(era_path + "*_q_*" + season +".nc")[0]).q.values
    r   = xr.open_dataset(glob.glob(era_path + "*_r_*"+ season +".nc")[0]).r.values
    tcc = xr.open_dataset(glob.glob(era_path + "*tcc*"+ season +".nc")[0]).tcc.values
    sp  = xr.open_dataset(glob.glob(era_path + "*sp*"+ season +".nc")[0]).sp.values
    t2m = xr.open_dataset(glob.glob(era_path + "*t2m*"+ season +".nc")[0]).t2m.values
    assert np.shape(q) == np.shape(r) == np.shape(tcc) == np.shape(sp) == np.shape(t2m)
    
    nbr_times, nbr_lats, nbr_lon = np.shape(q)
    
    train = []
    true  = tcc[:, np.newaxis] # is wrong now
    
    
    for i in range(nbr_times):
        one_timestep = np.array([q[i], r[i], sp[i], t2m[i] ])
        train.append(one_timestep)

    return np.array(train), true

In [4]:
train, true = load_data()

In [5]:
train.mean(axis = 0).mean(axis=1).mean(axis=1)

array([8.1528546e-03, 7.2762016e+01, 9.8931422e+04, 2.8904562e+02],
      dtype=float32)

In [6]:
train.shape

(2184, 4, 35, 60)

In [7]:
def train_test_split(data, test_split = 0.2):
    assert test_split < 1, 'test split is given as a decimal number, choose a number between 0, 1'
    indx = int(len(data)*(1-test_split))
    # returns train test
    return data[:indx], data[indx:]

In [8]:
def batch_normalize(data, seq_len = 4): 
    samples, metvars, lats, lons = data.shape
    #.mean(axis = 0).mean(axis=1).mean(axis=1)
    
    normalized = np.zeros((samples, metvars, lats, lons))
    means   = np.zeros(metvars)
    storage = np.zeros(metvars)
    
    for i in range(metvars):
        raveled = data[:, i, :, :].reshape(-1)
        m = raveled.mean()
        s = raveled.std()
        normalized[:, i, :, :] =  (data[:, i, :, :] - m)/s
        
    samples, metvars, lats, lons = normalized.shape
    assert seq_len % 4 == 0
    
    new_samples = int(samples/seq_len)
    normalized  = normalized.reshape( (new_samples, seq_len, metvars, lats, lons ) )
    
    return normalized, means, storage

In [9]:
from tensorflow.keras.layers import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

In [10]:
x, means_x, storage_x = batch_normalize(train)

In [11]:
y, means_y, storage_y = batch_normalize(true)

In [12]:
x.shape

(546, 4, 4, 35, 60)

In [13]:
# normalized, means, storage

## Normalize

In [14]:
#units = normalized.shape

In [15]:
#cell = keras.layers.ConvLSTM2DCell(filters, kernel_size, strides=(1, 1), padding='same', data_format=None,
#                            dilation_rate=(1, 1), activation='tanh', recurrent_activation='hard_sigmoid', 
#                            use_bias=True, kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', 
#                            bias_initializer='zeros', unit_forget_bias=True, kernel_regularizer=None,
#                            recurrent_regularizer=None, bias_regularizer=None, 
#                            kernel_constraint=None, recurrent_constraint=None, bias_constraint=None, 
#                            dropout=0.0, recurrent_dropout=0.0)

In [16]:
def calcMB( l ):
    """Returnes the megabytes of memory this data uses.
    Use this to detemine the batch..?
    """
    size = np.array(l.shape)
    return np.prod(size)*8
     

In [17]:
x.shape, y.shape

((546, 4, 4, 35, 60), (546, 4, 1, 35, 60))

In [18]:
#x = x.reshape(-1, 2184, 4, 35, 60)
#y = y.reshape(-1, 2184, 1, 35, 60)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization

#from keras.layers import keras.layers.

model1 =  Sequential()

# Begin with 2D convolutional LSTM layer
model1.add(ConvLSTM2D(filters = 256, kernel_size = (3, 3),
                     input_shape = (4, 4, 35, 60),
                     padding = 'same', return_sequences=True, data_format='channels_first')) 

model1.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (3, 3),
                                  #input_shape = (256, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

model1.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
model1.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3), # (1, 1)
                                  #input_shape=(None, 40, 40, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 


In [20]:
#for i in model.get_config()

In [21]:
def plot(history):
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()
    return

In [22]:
#from keras.utils import plot_model
#plot_model(model, to_file=save_dir+'model.png')

In [23]:
!pip3 install datetime --user

In [ ]:
# Prepare model for training
model1.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)
import datetime
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


history = model1.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 4,
                    validation_split = 0.2,
                    callbacks=[tensorboard_callback]
                    )

plot(history)

Train on 436 samples, validate on 110 samples
Epoch 1/4


In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
history

In [2]:
from tensorkeras.models import Sequential
from keras.layers import ConvLSTM2D
#from keras.layers.normalization import BatchNormalization

#from keras.layers import keras.layers.

def build_model(filters = [256, 128, 64], kernal = [5, 5, 3], input_shape = (4, 4, 35, 60)):
    model =  Sequential()

    # Begin with 2D convolutional LSTM layer
    for i, filter_ in enumerate( filters ):
        k_s = kernal[i]
        
        if i == 0:
            model.add(ConvLSTM2D(filters = filter_, kernel_size = (k_s, k_s),
                                 input_shape = input_shape,
                                 padding = 'same', return_sequences=True, data_format='channels_first')) 
        else:
            model.add(ConvLSTM2D(filters = filter_, kernel_size =  (k_s, k_s),
                                              #input_shape = (256, 35, 60, 1),
                                              padding='same', return_sequences=True,  data_format='channels_first')) 

    # Add 3x hidden 2D convolutions LSTM layers    
    # Begin with 2D convolutional LSTM layer
    # Blir dette rett da..?
    model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(1, 1),
                                      #input_shape=(None, 40, 40, 1),
                                      padding='same', return_sequences=True,  data_format='channels_first')) 
    return model

ModuleNotFoundError: No module named 'keras'

In [1]:


model =  Sequential()

# Begin with 2D convolutional LSTM layer
model.add(ConvLSTM2D(filters = 256, kernel_size = (5, 5),
                     input_shape = (4, 4, 35, 60),
                     padding = 'same', return_sequences=True, data_format='channels_first')) 

model.add(keras.layers.ConvLSTM2D(filters = 128, kernel_size = (5, 5),
                                  #input_shape = (256, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

model.add(keras.layers.ConvLSTM2D(filters=64, kernel_size=(3, 3),
                                  #input_shape=(None, 35, 60, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Add 3x hidden 2D convolutions LSTM layers    
# Begin with 2D convolutional LSTM layer
model.add(keras.layers.ConvLSTM2D(filters=1, kernel_size=(3, 3),
                                  #input_shape=(None, 40, 40, 1),
                                  padding='same', return_sequences=True,  data_format='channels_first')) 

# Prepare model for training
model.compile(
      loss = "mse",
      metrics = ["mae"],
      optimizer = "adam"
)

history2 = model.fit(x,
                    y,
                    batch_size = 32,
                    epochs = 40,
                    validation_split = 0.2, 
                    callbacks=[tensorboard_callback]
                    )



ModuleNotFoundError: No module named 'keras'

In [ ]:
history2

In [ ]:
plot(history2)

In [ ]:
from keras.models import model_from_json


def save_json(model, m = "model.json"):    
    """ Storing a config / architecture of the model."""
    # serialize model to JSON
    model_json = model.to_json()
    with open(m, "w") as json_file:
        json_file.write(model_json)
    return

def load_model(m = 'model.json'):
    """ Loading a stored config/ architecture  to a model. """
    # load json and create model
    json_file = open(m, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    return loaded_model

In [ ]:
model_storage = '/home/hanna/MS/stored_models/'

In [ ]:
save_json( model, model_storage + "example_keras.json" )

In [ ]:
model = load_model(  model_storage + "example_keras.json" )

In [ ]:
model.save_weights(model_storage + "example_keras.h5")

In [ ]:
model.load_weights(model_storage + "example_keras.h5")

## Implement your own/ Custumize metrics  